In [1]:
import matplotlib.pyplot as plt
import numpy  as np
import pandas as pd
import math

from scipy import optimize
from scipy import stats

from scipy.stats import norm


from scipy.optimize import leastsq

from scipy.stats._distn_infrastructure import rv_continuous

from scipy import special as sp
import binopt

%matplotlib inline
# plt.style.use('physics')

plt.rcParams['axes.grid'       ]  = False
plt.rcParams['xtick.labelsize' ]  = 14
plt.rcParams['ytick.labelsize' ]  = 14
plt.rcParams['axes.labelsize'  ]  = 14
plt.rcParams['legend.fancybox' ]  = False

pd.options.mode.chained_assignment = None

In [2]:
# low mass
df = pd.read_hdf('BoundaryOpt_Feb2019_Bkg-ggh.h5') # 

In [3]:
np.unique(df['sample'])

array(['dipho', 'ggh', 'gjet', 'qcd', 'sign'], dtype=object)

In [4]:
df.head()

,dijet_LeadJPt,dijet_SubJPt,dijet_abs_dEta,dijet_Mjj,dijet_dipho_dphi_trunc,dijet_dipho_dphi,leadPho_PToM,sublPho_PToM,dipho_PToM,dipho_mva,dijet_mva,dipho_dijet_MVA,dijet_minDRJetPho,dijet_Zep,dijet_dphi,dipho_mass,weight,sample,Y
0,83.956253,54.704861,3.972195,487.604523,2.941600,3.027010,1.471198,0.344576,1.795790,0.695890,0.304356,0.512101,2.894457,1.650994,0.756914,86.893074,-0.009839,sign,1.0
1,116.253372,66.077904,4.222458,734.435425,2.912361,2.912361,0.511307,0.486596,0.098913,0.247714,0.011805,0.366422,0.664184,3.519514,2.834856,90.761864,0.012357,sign,1.0
2,186.902924,63.367134,7.043717,3686.878418,2.941600,3.041730,0.893491,0.429932,1.003458,0.591548,0.913009,0.920536,2.726236,1.316373,3.128963,86.976921,0.010753,sign,1.0
3,59.541355,56.464218,8.126570,3372.178955,2.941600,3.028491,0.629867,0.454643,1.064794,0.600556,0.891809,0.934031,3.959622,0.304246,1.109837,90.646851,0.007024,sign,1.0
4,94.403702,54.080784,3.321884,374.739227,2.902826,2.902826,0.897126,0.706207,1.591184,0.429133,0.313785,0.420558,2.488332,0.546378,1.334557,89.075813,0.000859,sign,1.0


In [5]:
# VBF preselection cut for training
def vbf_presel(data):
    return (
#         data["weight"] * (
        (data["leadPho_PToM"       ]> 0.47)&  # SM cut
        (data["sublPho_PToM"       ]> 0.28)&
        (data["dijet_LeadJPt"      ]> 40     )& 
        (data["dijet_SubJPt"       ]> 30     )&
        (data["dijet_Mjj"          ]> 250    )&  # 250
        (data["dipho_mass"         ]> 65    )&  # 100  (low mass 55 or 65??)
        (data["dipho_mass"         ]< 120    ))  # 180

df = df[vbf_presel(df)]
df.columns.values

array(['dijet_LeadJPt', 'dijet_SubJPt', 'dijet_abs_dEta', 'dijet_Mjj',
       'dijet_dipho_dphi_trunc', 'dijet_dipho_dphi', 'leadPho_PToM',
       'sublPho_PToM', 'dipho_PToM', 'dipho_mva', 'dijet_mva',
       'dipho_dijet_MVA', 'dijet_minDRJetPho', 'dijet_Zep', 'dijet_dphi',
       'dipho_mass', 'weight', 'sample', 'Y'], dtype=object)

In [6]:
df_bkgs = df[
    (df['sample'] == 'qcd'  ) |
    (df['sample'] == 'dipho') |
    (df['sample'] == 'gjet') ]
    
df_sign = df[
    (df['sample'] == 'sign')
]
df_ggh = df[df['sample']=='ggh']

In [7]:
I  =  np.concatenate((df_sign['sample'],df_ggh['sample'],df_bkgs['sample']))

In [8]:
# 2017 low mass ( vbf-ggh-allx0_noQcd )
import collections 
print "number of classes (samples) inside the dataset ..."
for p in collections.Counter(I):
    print "%10s nevent = %10.2f" % ( p, df[df['sample']==p].shape[0])

number of classes (samples) inside the dataset ...
      gjet nevent =    1186.00
       qcd nevent =       2.00
       ggh nevent =    3465.00
     dipho nevent =  368211.00
      sign nevent =    7213.00


In [9]:
rng = np.random.RandomState(15)  # deterministic random data

# s = df_sign.dijet_bdt
# b = df_bkgs.dijet_bdt
s = df_sign.dipho_dijet_MVA
b = df_bkgs.dipho_dijet_MVA

ms = df_sign.dipho_mass
mb = df_bkgs.dipho_mass


ws = df_sign.weight
wb = df_bkgs.weight * 1.38 # 0.679 (wrong result with loose cut for gj and jj)

X = np.concatenate([s,b])
Y = np.concatenate([np.ones(s.shape[0]), np.zeros(b.shape[0])])
W = np.concatenate([ws,wb])
M = np.concatenate([ms,mb])

we_s, x = np.histogram(s, bins=50, range=[-1,1], weights=ws**2)
we_b, _ = np.histogram(b, bins=50, range=[-1,1], weights=wb**2)

he_s, _ = np.histogram(s, bins=50, range=[-1,1], weights=ws)
he_b, _ = np.histogram(b, bins=50, range=[-1,1], weights=wb)

In [10]:
import binopt
help(binopt.optimize_bin)
# help(core.optimize_bin)

Help on class optimize_bin in module binopt.core:

class optimize_bin(binner_base)
 |  Optimse bining of lableled data.
 |  
 |  Method resolution order:
 |      optimize_bin
 |      binner_base
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, nbins, range, drop_last_bin=True, fix_upper=True, fix_lower=False, use_kde_density=True, W=None, mass=None)
 |      Init.
 |  
 |  binned_score(self, x, breg=None)
 |      Binned score.
 |  
 |  binned_score_cdf(self, x, breg=None)
 |  
 |  binned_score_density(self, x, breg=None)
 |      Binned score after KDE estimation of the distributions.
 |  
 |  binned_score_fit(self, x, breg=None, nsig=1)
 |      Input should contain a resonance of some sort.
 |  
 |  binned_stats(self, x)
 |      Binned score.
 |  
 |  boundary_scan_2d(self, fig=None, title_fmt='.2f', max_n_ticks=5, steps=0.01, label='parameter_scan')
 |      Fit result displayed on matrix.
 |  
 |  cost_fun(self, x, breg=None, lower_bound=None, upper_bou

## Boundary Optimization with range [-1,1] (scale factor = 1.38)

## 1 bin optimization

In [11]:
import binopt
# 1 bins
binner = binopt.optimize_bin(nbins=1, range=[-1,1], # SM: nbins = 1
                             drop_last_bin=True, fix_upper=True, 
                             fix_lower=True, use_kde_density=False)

In [11]:
# 8 May 2019 (20:35 after removing some parameter "mass" from "fit" function)
import core
# 1 bins
binner = core.optimize_bin(nbins=1, range=[-1,1], # SM: nbins = 1
                             drop_last_bin=True, fix_upper=True, 
                             fix_lower=True, use_kde_density=False)

In [12]:
# 9 May 2019 (12:30 : use core.py in installed packag
binner.fit(X, Y,sample_weights=W, mass=M, method="Nelder-Mead", breg=5, fom="AMS3")

breg =  5
cost function :  [0.]
     function :  [1.] 1 None
[1 1 1 ... 1 1 1]


/cvmfs/sft.cern.ch/lcg/views/LCG_95a/x86_64-centos7-gcc7-opt/lib/python2.7/site-packages/scipy/optimize/_minimize.py:518: RuntimeWarning: Method Nelder-Mead cannot handle constraints nor bounds.
  RuntimeWarning)


ValueError: operands could not be broadcast together with shapes (369399,) (0,) 